# Démo projet Combinatoire

In [2]:
from IPython.display import HTML
import Rules as R
import Grammars as G
import GrammarsC as GC
import CRules as CR


In [3]:
class IncorrectGrammar(Exception):

    def __init__(self, message):
        super(IncorrectGrammar, self).__init__(message)
        
'''Vérifie que tous les non-terminaux apparaissant dans 
les règles sont bien définis dans la grammaire '''
def check_defined_rules (grammar):
    for rule_id in grammar:
        rule = grammar[rule_id]
        if isinstance(rule, R.ConstructorRule):
            fst, snd = rule._parameters
            if not (fst in grammar):
                raise IncorrectGrammar(fst+" rule not defined in grammar")
            if not ( snd in grammar):
                raise IncorrectGrammar(snd+" rule not defined in grammar")

            
# Sets grammar for each rule, computes valuations, checks grammar
def init_grammar (grammar):

    for rule_id in grammar:
        grammar[rule_id]._set_grammar(grammar)

    check_defined_rules(grammar)

    ''' Pour le calcul des valuations on a besoin de comparer 
    les résultats de l'itération courante avec ceux de l'itération précédente,
    on stocke donc les valuations calculées dans les dictionnaires val_before 
    et val_after'''

    val_before, val_after = {}, {}

    for rule in grammar.values():
        if isinstance(rule, R.ConstructorRule):
            val_before[rule] = rule.valuation() 
            val_after [rule] = 0

    # Tant que l'on n'a pas atteint le point fixe, on met à jour les valuations 
    while (val_before != val_after):
        val_before = val_after.copy()
        for rule in val_after:
            rule._update_valuation()
            val_after[rule] = rule.valuation()
   
    # Si on atteint un point fixe pour lequel une des règles à une valuation infine,
    # la grammaire est mal construite
    for rule_id in grammar:
        if grammar[rule_id].valuation() == float('inf'):
            raise IncorrectGrammar("Rule "+rule_id+" is incorrect (inf valuation)")

   
# Récupérer un dictionnaire associant à chaque règle de la 
#  grammaire sa valuation
def get_valuation(gram):
    d = {}
    for rule_name in gram:
        d[rule_name] = gram[rule_name].valuation()
    return d



On récupère un dictionnaire qui contient toutes les grammaires définies,
puis on vérifie que tous les non-terminaux de la grammaire sont bien définis (fonction check_defined_rules), on calcule les valuations et on initialise les grammaires (fonction init_grammar)

In [4]:
grammars = G.grammars

for g in grammars:
    check_defined_rules(grammars[g][0])
    init_grammar(grammars[g][0])


In [5]:
''' EFFECTUE LES TEST POUR i = 0, ...., n - 1'''
def tests (name, gram, rule_init, card_fun, n, valuation = []):
    print ("\nTests on "+name)
    print("\n")
    #G.print_grammar(gram)
    rule = gram[rule_init]
    try:
        
        # V A L U A T I O N 

        if len(valuation) != 0:
            print ("Valuation:")
            assert (valuation == get_valuation(gram))
            display(HTML("<p style=\"color:green;\">Passed</p>"))

        # C A R D I N A L I T Y

        print ("\nCardinality:")
        for i in range(n):
            count = rule.count(i)
            assert (count == card_fun(i))
            assert (count == len(rule.list(i)))
        display(HTML("<p style=\"color:green;\">Passed</p>"))

        # R A N K

        try:
            print ("\nRank:")
            for j in range(n):
                assert([rule.rank(i) for i in rule.list(j)] == list(range(rule.count(j))))
            display(HTML("<p style=\"color:green;\">Passed</p>"))
        except NotImplementedError:
            print ("Rank not available for this grammar")
            
        # U N R A N K

        print("\nUnrank:")
        for j in range(n):
            assert(rule.list(j) == [rule.unrank(j, i) for i in list(range(rule.count(j)))])
        display(HTML("<p style=\"color:green;\">Passed</p>"))

        # V A L U A T I O N  &  C O U N T

        ''' La valuation correspond au plus petit mot qu'une règle peut produire :
        on vérifie donc pour chaque règle que la valeur de la valuation calculée correspond au plus 
        petit produit par la règle'''
        
        print("\nValuation & Count:")
        valuation = get_valuation(gram)
        for rule_name in gram:
            i = 0
            while gram[rule_name].count(i) == 0:
                i += 1
            assert(i == valuation[rule_name])
        display(HTML("<p style=\"color:green;\">Passed</p>"))
        
    except AssertionError:
        display(HTML("<p style=\"color:red;\">Not passed</p>"))


In [6]:
Test_Size = 10

for g in grammars:
    tests(g, grammars[g][0], grammars[g][1], grammars[g][2], Test_Size)



Tests on ABCPalindrome



Cardinality:



Rank:



Unrank:



Valuation & Count:



Tests on ThreeGram



Cardinality:


...


Valuation & Count:



Tests on TwoGram



Cardinality:



Rank:



Unrank:



Valuation & Count:


On exécute également les tests sur les grammaires condensées

In [7]:
gramC = GC.grammarsC

for g in gramC:
    gramC[g][0] = CR.dvp_gram(gramC[g][0])
    init_grammar(gramC[g][0])


for g in gramC:
    check_defined_rules(gramC[g][0])
    tests(g, gramC[g][0], gramC[g][1], gramC[g][2], Test_Size)


Tests on ABCPalindrome



Cardinality:



Rank:



Unrank:



Valuation & Count:



Tests on ThreeGram



Cardinality:


...


Valuation & Count:



Tests on TwoGram



Cardinality:



Rank:



Unrank:



Valuation & Count:


On teste que les fonctions retournent la même chose pour les mêmes grammaires exprimées sous forme développée
ou sous forme condensée

In [10]:
def tests_equal (name, gram1, gram2, rule_init, card_fun, n):
    print ("\nTests on "+name)
    print("\n")
    rule1 = gram1[rule_init]
    rule2 = gram2[rule_init]
    try:  

        # C A R D I N A L I T Y

        print ("\nCardinality:")
        for i in range(n):
            assert (rule1.count(i) == rule2.count(i))
        display(HTML("<p style=\"color:green;\">Passed</p>"))

        
        # L I S T
        
        print ("\nList:")
        for i in range(n):
            assert (rule1.list(i) == rule2.list(i))
        display(HTML("<p style=\"color:green;\">Passed</p>"))

            
        # R A N K

        try:
            print ("\nRank:")
            for j in range(n):
                assert([rule1.rank(i) for i in rule1.list(j)] == [rule2.rank(i) for i in rule2.list(j)])
            display(HTML("<p style=\"color:green;\">Passed</p>"))
        except NotImplementedError:
            print ("Rank not available for this grammar")
            
        # U N R A N K

        print("\nUnrank:")
        for j in range(n):
            assert([rule1.unrank(j, i) for i in list(range(rule1.count(j)))] == 
                   [rule2.unrank(j, i) for i in list(range(rule2.count(j)))])
        display(HTML("<p style=\"color:green;\">Passed</p>"))
        
    except AssertionError:
        display(HTML("<p style=\"color:red;\">Not passed</p>"))


In [14]:
for g in gramC:
    tests_equal(g, gramC[g][0], grammars[g][0], gramC[g][1], gramC[g][2], Test_Size)


Tests on ABCPalindrome



Cardinality:



List:



Rank:



Unrank:



Tests on ThreeGram



Cardinality:


...


Unrank:



Tests on TwoGram



Cardinality:



List:



Rank:



Unrank:


## Exemples

### Mots sur l'alphabet AB

In [19]:
r = grammars["ABGram"][0]["AB"]
print(r.list(3))
print(r.count(3))
print(r.rank('BBA'))
print(r.unrank(3, 4))
print(r.random(10))

['AAA', 'AAB', 'ABA', 'ABB', 'BAA', 'BAB', 'BBA', 'BBB']
8
6
BAA
BBAAAABABB


### Arbres

In [25]:
r = grammars["treeGram"][0]["Tree"]
print(r.list(4))
print(r.count(4))
print(r.unrank(4, 2))
print(r.random(10))

[(Leaf, (Leaf, (Leaf, Leaf))), (Leaf, ((Leaf, Leaf), Leaf)), ((Leaf, Leaf), (Leaf, Leaf)), ((Leaf, (Leaf, Leaf)), Leaf), (((Leaf, Leaf), Leaf), Leaf)]
5
((Leaf, Leaf), (Leaf, Leaf))
((Leaf, ((Leaf, (Leaf, (Leaf, Leaf))), ((Leaf, Leaf), (Leaf, Leaf)))), Leaf)


### Mots de Fibonacci

In [27]:
r = grammars["fiboGram"][0]["Fib"]
print(r.list(4))
print(r.count(4))
print(r.unrank(4, 2))
print(r.rank('BABA'))
print(r.random(10))

['AAAA', 'AAAB', 'AABA', 'ABAA', 'ABAB', 'BAAA', 'BAAB', 'BABA']
8
AABA
7
AABAAABAAB


### Mots bien parenthésés

In [36]:
r = grammars["DyckGram"][0]["Dyck"]
print(r.list(6))
print(r.list(7))
print(r.count(6))
print(r.unrank(6, 0))
print(r.random(10))

['((()))', '(()())', '()(())', '(())()', '()()()']
[]
5
((()))
()((()()))


### Palindromes sur A, B

In [39]:
r = grammars["ABPalindrome"][0]["Pal"]
print(r.list(5))
print(r.count(5))
print(r.rank('AABAA'))
print(r.unrank(5, 3))
print(r.random(10))

['AAAAA', 'AABAA', 'ABABA', 'ABBBA', 'BAAAB', 'BABAB', 'BBABB', 'BBBBB']
8
1
ABBBA
ABABAABABA


### Palindromes sur A, B, C

In [41]:
r = grammars["ABCPalindrome"][0]["Pal"]
print(r.list(5))
print(r.count(5))
print(r.rank('ACBCA'))
print(r.unrank(5, 3))
print(r.random(10))

['AAAAA', 'AABAA', 'AACAA', 'ABABA', 'ABBBA', 'ABCBA', 'ACACA', 'ACBCA', 'ACCCA', 'BAAAB', 'BABAB', 'BACAB', 'BBABB', 'BBBBB', 'BBCBB', 'BCACB', 'BCBCB', 'BCCCB', 'CAAAC', 'CABAC', 'CACAC', 'CBABC', 'CBBBC', 'CBCBC', 'CCACC', 'CCBCC', 'CCCCC']
27
7
ABABA
AABBAABBAA


### Mots avec le meme nombre de A que de B

In [44]:
r = grammars["EqualGram"][0]["S"]
print(r.list(6))
print(r.count(5))
print(r.count(6))
print(r.rank('AABABB'))
print(r.unrank(5, 3))
print(r.random(10))

['ABABAB', 'ABABBA', 'ABAABB', 'ABBAAB', 'ABBABA', 'ABBBAA', 'AABBAB', 'AABBBA', 'AABABB', 'AAABBB', 'BAABAB', 'BAABBA', 'BAAABB', 'BABAAB', 'BABABA', 'BABBAA', 'BBAAAB', 'BBAABA', 'BBABAA', 'BBBAAA']
0
20


NotImplementedError: To call rank, provide function as arg